# 本代码为：https://zhuanlan.zhihu.com/p/28196873  里代码

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# 构建基础RNN单元

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=128) # state_size = 128
print(cell.state_size) # 128

128


In [5]:
print(cell.compute_output_shape())

TypeError: compute_output_shape() missing 1 required positional argument: 'input_shape'

In [7]:
# 输入input：x1 和 初始隐藏单元值： h0， 利用 __calll__调用，前进一个时间步子

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态，形状为(batch_size, state_size)
output, h1 = cell.__call__(inputs, h0) #调用call函数

print(h1.shape) # (32, 128)

(32, 128)


In [9]:
# 构建基础LSTM单元

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)

# 两个隐藏层
print(lstm_cell.state_size.c)
print(lstm_cell.state_size.h)

128
128


In [11]:
# 单步运行一次

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size

h0 = lstm_cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = lstm_cell.__call__(inputs, h0)

print(h1.h)  # shape=(32, 128)
print(h1.c)  # shape=(32, 128)

Tensor("basic_lstm_cell/Mul_2:0", shape=(32, 128), dtype=float32)
Tensor("basic_lstm_cell/Add_1:0", shape=(32, 128), dtype=float32)


In [3]:
import tensorflow as tf
import numpy as np

# 一次执行多步 ， TensorFlow提供了一个tf.nn.dynamic_rnn函数，使用该函数就相当于调用了n次call函数
# 即通过{h0,x1, x2, …., xn}直接得{h1,h2…,hn}

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)

# inputs: shape = (batch_size, time_steps, input_size) 

inputs = tf.placeholder(np.float32, shape=(32,50,256))

# cell: RNNCell
# initial_state: shape = (batch_size, cell.state_size)。初始状态。一般可以取零矩阵


outputs, states = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)

# 此时，得到的outputs就是time_steps步里所有的输出。它的形状为(batch_size, time_steps, cell.output_size)。
# state是最后一步的隐状态，它的形状为(batch_size, cell.state_size)

In [4]:
print(outputs.shape)
print(states.c)
print(states.h)

(32, 50, 128)
Tensor("rnn/while/Exit_3:0", shape=(32, 128), dtype=float32)
Tensor("rnn/while/Exit_4:0", shape=(32, 128), dtype=float32)


In [5]:
# 多层RNN

import tensorflow as tf
import numpy as np

# 每调用一次这个函数就返回一个BasicRNNCell
def get_a_cell():
    return tf.nn.rnn_cell.BasicRNNCell(num_units=128)

# 用tf.nn.rnn_cell MultiRNNCell创建3层RNN
cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(3)]) # 3层RNN

# 得到的cell实际也是RNNCell的子类
# 它的state_size是(128, 128, 128)
# (128, 128, 128)并不是128x128x128的意思
# 而是表示共有3个隐层状态，每个隐层状态的大小为128

print(cell.state_size) # (128, 128, 128)
# 使用对应的call函数

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = cell.call(inputs, h0)

print(h1) # tuple中含有3个32x128的向量

'''
在BasicRNNCell中，output其实和隐状态的值是一样的。因此，我们还需要额外对输出定义新的变换，才能得到图中真正的输出y。
'''

(128, 128, 128)
(<tf.Tensor 'cell_0/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_1/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_2/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>)
